In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from linearmodels.iv import IV2SLS
import IV_estimation as IV


In [2]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]
data.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [3]:
# Copy the dataframe
df2 = data.copy()

In [4]:
# Creating dummies for each segment
df2 = pd.get_dummies(df2, columns=['Segment'], drop_first=True)

# Creating dummies for each year
df2 = pd.get_dummies(df2, columns=['Year'], drop_first=True)

# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [5]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

In [6]:
y = df2['log_market_share']
x = df2[['const']]
dummies = df2[['Segment_B', 'Segment_C', 'Segment_D', 'Segment_E', 'Segment_F', 'Segment_M', 'Segment_J',
                'Year_2014', 'Year_2015', 'Year_2016', 'Year_2017', 'Year_2018', 'Year_2019', 'Year_2020', 'Year_2021', 'Year_2022', 'Year_2023',
                'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['HP', 'Range', 'Chargetime']]

In [7]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='unadjusted')

In [8]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -0.5555
Estimator:                    IV-2SLS   Adj. R-squared:                -0.6496
No. Observations:                 334   F-statistic:                    101.74
Date:                Thu, Apr 25 2024   P-value (F-stat)                0.0000
Time:                        13:45:05   Distribution:                 chi2(19)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.2507     1.3198    -3.9783     0.0001     -7.8375     -2.6639
Segment_B     -0.7875     0.8791    -0.8959     0.3703     -2.5105      0.9354
Segment_C     -0.8217     1.0856    -0.7569     0.4491     -2.9494      1.3060
Segment_D     -2.9660     1.7620    -1.6833     0.0923     -6.4195      0.4874
Segment_E     -8.5447     2.8757    -2.9714     0.0030     -14.181     -2.9085
Segment_F     -13.945     4.5406    -3.0712     0.0021     -22.844     -5.0456
Segment_M     -4.6895     1.4999    -3.1265     0.0018     -7.6294     -1.7497
Segment_J     -1.8498     3.2034    -0.5775     0.5636     -8.1283      4.4286
Year_2014      0.7164     1.6769     0.4272     0.6692     -2.5703      4.0031
Year_2015     -0.5247     1.6170    -0.3245     0.7456     -3.6940      2.6446
Year_2016     -1.0208     1.6063    -0.6355     0.5251     -4.1690      2.1275
Year_2017     -1.3741     1.5829    -0.8681     0.3854     -4.4765      1.7283
Year_2018     -2.1959     1.5514    -1.4155     0.1569     -5.2365      0.8447
Year_2019     -2.5720     1.4632    -1.7578     0.0788     -5.4398      0.2957
Year_2020     -3.3571     1.3871    -2.4202     0.0155     -6.0758     -0.6384
Year_2021     -4.4595     1.3889    -3.2107     0.0013     -7.1817     -1.7372
Year_2022     -5.3581     1.4629    -3.6626     0.0002     -8.2254     -2.4908
Year_2023     -5.5117     1.4044    -3.9246     0.0001     -8.2643     -2.7592
China          1.2606     0.8823     1.4287     0.1531     -0.4688      2.9899
Price       1.641e-05  5.028e-06     3.2639     0.0011   6.557e-06   2.627e-05
==============================================================================

Endogenous: Price
Instruments: HP, Range, Chargetime
Unadjusted Covariance (Homoskedastic)
Debiased: False
"""

# Test

In [9]:
df1 = pd.read_csv('dataset.csv')
Nobs=df1['ID'].count()
df1['const']=np.ones((Nobs,1))
df1 = df1[df1['Market share'] != 0]
#df1['HP_instrument'] = 0
df1.head(20)

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [10]:
models = df1['Model'].unique()
HP = df1['HP']
print(models)

['U5' 'U6' 'e-tron' 'e-tron GT' 'Q4 e-tron' 'Q8 e-tron' 'I3' 'i4' 'i5'
 'I7' 'iX' 'iX1' 'iX3' 'Atto 3' 'Dolphin' 'Han' 'Seal' 'Tang' 'Berlingo'
 'C4' 'ë-C4 X' 'e-SpaceTourer' 'Jumpy' 'Born' 'Spring' '3 Crossback' '500'
 'Ocean' 'Mustang Mach-E' 'E' 'e:Ny1' 'E-HS9' 'Ioniq' 'Ioniq 5' 'Ioniq 6'
 'Kona' 'e-JS4' 'I-Pace' 'Avenger' 'EV6' 'EV9' 'Niro' 'Soul' 'RZ' 'UX'
 'Euniq' 'Euniq5' 'Euniq6' 'MIFA 9' 'MX-30' 'EQA' 'EQB' 'EQC' 'EQE'
 'EQE SUV' 'EQS' 'EQS SUV' 'EQV' '4' '5' 'Marvel R' 'ZS' '3-dørs'
 'Cabriolet' 'EL6' 'ES8' 'ET5' 'ET7' 'Ariya' 'Leaf' 'Combo' 'Corsa'
 'Mokka' 'Zafira' '2008' '208' 'Expert' 'Rifter' 'Traveller' '2' 'Taycan'
 'Megane' 'Zoe' 'Mii' 'Citigo' 'Enyaq iV' 'Fortwo' 'Korando' 'Solterra'
 'Model 3' 'Model S' 'Model X' 'Model Y' 'bZ4X' 'Proace City Verso'
 'Proace Verso' 'Golf' 'ID.3' 'ID.4' 'ID.5' 'ID.7' 'ID.Buzz' 'up!' 'C40'
 'EX30' 'XC40' 'Free']


In [11]:
def HP_instrument1(df1):
    z = 0
    models = df1['Model'].unique()
    for model in models:
        for i in df1:
            if df1[model] == df1['Model'][i]:
                z = z
            else:
                z += df1['HP'][i]
    df1['HP_instrument'] = z            

In [12]:
def create_instrument(df1, instrument):
    df1[instrument+'_instrument'] = 0
    for index, row in df1.iterrows():
        current_model = row['Model']
        hp_sum_except_current = df1[df1['Model'] != current_model][instrument].sum()
        df1.at[index, instrument+'_instrument'] = hp_sum_except_current


In [13]:
create_instrument(df1, 'Chargetime')

In [14]:
df1

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const,Chargetime_instrument
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CH,257,1.0,11308
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CH,183,1.0,11308
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CH,177,1.0,11308
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CH,3,1.0,11376
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0,11325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1154,189,2023,0.000065,Volvo,EX30,475,368245.000,268,28,SUV,B,SE,4,1.0,11382
1163,190,2021,0.014652,Volvo,XC40,457,462060.600,402,28,SUV,C,SE,363,1.0,11326
1164,190,2022,0.033310,Volvo,XC40,457,416263.400,402,28,SUV,C,SE,1020,1.0,11326
1165,190,2023,0.031752,Volvo,XC40,457,439266.600,402,28,SUV,C,SE,1965,1.0,11326
